## Get Summaries for Visualizing Segment Race Times (and proportions)

#### Categorize And Clean Data

In [1]:
# Try to use Seaborn/Matplotlib to replicate another cool viz from Cole Nusaumbber Knaflic's book Storytelling with Data.
# Inspiration from :
# https://github.com/adamribaudo/storytelling-with-data-ggplot
# Using code from the excellent:
# https://github.com/empathy87/storytelling-with-data

## Import libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib import transforms
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FixedLocator, FixedFormatter
from pandas import DataFrame

%matplotlib inline

In [2]:
df_Apr = pd.read_csv ('..\Data\April_2022.csv'  )
df_Apr['Month'] = "Apr"

df_Oct= pd.read_csv ('..\Data\October_2022.csv'  )
df_Oct['Month'] = "Oct"

In [3]:
df = pd.concat  ( [  df_Apr , df_Oct ] )

#### Temporary fix up - move this code to Scratch Notebook

In [4]:
df['0K_5K_SECTION_TIME']   = df['5K_MINS']
df['5K_10K_SECTION_TIME']  = df['10K_MINS']  - df['5K_MINS']
df['10K_15K_SECTION_TIME'] = df['15K_MINS']  - df['10K_MINS']
df['15K_20K_SECTION_TIME'] = df['20K_MINS']  - df['15K_MINS']

In [5]:
df.Month.value_counts()

Oct    13160
Apr    11572
Name: Month, dtype: int64

In [6]:
# Now Find the means for each section and Category
df.groupby (['Month' ] ).agg (
     num_runners=('RACE_NUMBER', 'count' )
    ,time_mins = ('CHIP_TIME_MINS', 'mean' )
).reset_index(drop=False)
#df_agg1.sort_values('CATEGORY')

,Month,num_runners,time_mins
0,Apr,11572,130.399274
1,Oct,13160,130.934758


In [7]:
df.loc [ df.CATEGORY.isna(), 'CATEGORY'] = 'NOT_GIVEN'
# There are two 'NOT_GIVEN' rows - drop them

df = df [ df.CATEGORY != 'NOT_GIVEN'  ]

#### Columns

In [8]:
df.columns

Index(['RACE_NUMBER', 'FIRST_NAME', 'SURNAME', 'GENDER', 'GENDER_POSITION',
       'CATEGORY', 'CATEGORY_POSITION', 'CLUB', '5K', '10K', '15K', '20K',
       'OVERALL_POSITION', 'CHIP_TIME', 'CHIP_POSITION', 'CHIP_TIME_MINS',
       '5K_MINS', '10K_MINS', '15K_MINS', '20K_MINS', 'min_per_km_course',
       'min_per_km_0_5', 'min_per_km_6_10', 'min_per_km_11_15',
       'min_per_km_16_20', 'min_per_km_21_FINISH', '5K_10K_SECTION_TIME',
       '10K_15K_SECTION_TIME', '15K_20K_SECTION_TIME', 'Month',
       '0K_5K_SECTION_TIME'],
      dtype='object')

#### Utility function

In [12]:
def flattenHierarchicalCol(col,sep = '_'):
    if not type(col) is tuple:
        return col
    else:
        new_col = ''
        for leveli,level in enumerate(col):
            if not level == '':
                if not leveli == 0:
                    new_col += sep
                new_col += level
        return new_col

#df_piv.columns = df_piv.columns.map(flattenHierarchicalCol)
#df_piv.head(10)


#### Recategorize

In [9]:
df['ANALYSIS_CATEGORY'] = "Normal"

In [10]:
#df.loc [df.CATEGORY.fillna ('No').str.contains ('OPEN')  , 'ENTRY_STATUS'] = "Elite"

boolIndex = np.logical_and ( df.CHIP_TIME_MINS <= 60 + 30  , df.GENDER== 'Male')
df.loc [ boolIndex , 'ANALYSIS_CATEGORY']  = 'Elite'

boolIndex = np.logical_and ( df.CHIP_TIME_MINS <= 60 + 40 , df.GENDER== 'Female')
df.loc [ boolIndex , 'ANALYSIS_CATEGORY']  = 'Elite'

#### Extract our rows from  the dataset.



In [18]:

boolIndex =    ( df.FIRST_NAME.str.contains ('Ellena') ) &\
               ( df.SURNAME.str.contains ( 'Green') )    &\
               (  df.Month == "Apr" )

df.loc [ boolIndex , 'ANALYSIS_CATEGORY']  = 'Us'


boolIndex =    ( df.FIRST_NAME.str.contains ('Ellena') ) &\
               ( df.SURNAME.str.contains ( 'Green') )    &\
               (  df.Month == "Oct" )

df.loc [ boolIndex , 'ANALYSIS_CATEGORY']  = 'Us'



boolIndex =    ( df.FIRST_NAME.str.contains ('John') ) &\
               ( df.SURNAME.str.contains ( 'Steedman') )&\
               (  df.Month == "Apr" )

df.loc [ boolIndex , 'ANALYSIS_CATEGORY']  = 'Us'


boolIndex =    ( df.FIRST_NAME.str.contains ('John') ) &\
               ( df.SURNAME.str.contains ( 'Steedman') )&\
               (  df.Month == "Oct" )

df.loc [ boolIndex , 'ANALYSIS_CATEGORY']  = 'Us'


In [19]:
df.ANALYSIS_CATEGORY.value_counts()

Normal    23261
Elite      1465
Us            4
Name: ANALYSIS_CATEGORY, dtype: int64

#### Improve the row labelling

In [26]:
df['Label'] = df.ANALYSIS_CATEGORY
df.loc[(df.Label == 'Us') & (df.GENDER == 'Male') , 'Label'] = 'John'
df.loc[(df.Label == 'Us') & (df.GENDER == 'Female') , 'Label'] = 'Ellena'

#### Extract the rows for us.

In [27]:
df_us = df [ df.ANALYSIS_CATEGORY == 'Us']
df_us

,RACE_NUMBER,FIRST_NAME,SURNAME,GENDER,GENDER_POSITION,CATEGORY,CATEGORY_POSITION,CLUB,5K,10K,...,min_per_km_11_15,min_per_km_16_20,min_per_km_21_FINISH,5K_10K_SECTION_TIME,10K_15K_SECTION_TIME,15K_20K_SECTION_TIME,Month,0K_5K_SECTION_TIME,ANALYSIS_CATEGORY,Label
4813,2338,John,Steedman,Male,2646,M45+,361,NaN,1900-01-01 00:24:30,1900-01-01 00:51:55,...,5.650000,6.160000,5.861807,27.416667,28.250000,30.800000,Apr,24.500000,Us,John
7601,13475,Ellena,Greenow,Female,3884,F40+,636,NaN,1900-01-01 00:36:25,1900-01-01 01:14:22,...,7.986667,8.220000,7.152620,37.950000,39.933333,41.100000,Apr,36.416667,Us,Ellena
2761,3371,John,Steedman,Male,1887,M50+,151,NaN,1900-01-01 00:23:49,1900-01-01 00:49:07,...,5.133333,5.716667,5.193622,25.300000,25.666667,28.583333,Oct,23.816667,Us,John
6001,15487,Ellena,Greenow,Female,2711,F40+,423,NaN,1900-01-01 00:29:50,1900-01-01 01:01:44,...,6.576667,6.800000,5.725133,31.900000,32.883333,34.000000,Oct,29.833333,Us,Ellena


#### Unpivot df_us  in order to do a line plot

In [39]:
df_us.columns

Index(['RACE_NUMBER', 'FIRST_NAME', 'SURNAME', 'GENDER', 'GENDER_POSITION',
       'CATEGORY', 'CATEGORY_POSITION', 'CLUB', '5K', '10K', '15K', '20K',
       'OVERALL_POSITION', 'CHIP_TIME', 'CHIP_POSITION', 'CHIP_TIME_MINS',
       '5K_MINS', '10K_MINS', '15K_MINS', '20K_MINS', 'min_per_km_course',
       'min_per_km_0_5', 'min_per_km_6_10', 'min_per_km_11_15',
       'min_per_km_16_20', 'min_per_km_21_FINISH', '5K_10K_SECTION_TIME',
       '10K_15K_SECTION_TIME', '15K_20K_SECTION_TIME', 'Month',
       '0K_5K_SECTION_TIME', 'ANALYSIS_CATEGORY', 'Label'],
      dtype='object')

In [51]:
help ( df_us.unstack )

Help on method unstack in module pandas.core.frame:

unstack(level: 'Level' = -1, fill_value=None) method of pandas.core.frame.DataFrame instance
    Pivot a level of the (necessarily hierarchical) index labels.
    
    Returns a DataFrame having a new level of column labels whose inner-most level
    consists of the pivoted index labels.
    
    If the index is not a MultiIndex, the output will be a Series
    (the analogue of stack when the columns are not a MultiIndex).
    
    Parameters
    ----------
    level : int, str, or list of these, default -1 (last level)
        Level(s) of index to unstack, can pass level name.
    fill_value : int, str or dict
        Replace NaN with this value if the unstack produces missing values.
    
    Returns
    -------
    Series or DataFrame
    
    See Also
    --------
    DataFrame.pivot : Pivot a table based on column values.
    DataFrame.stack : Pivot a level of the column labels (inverse operation
        from `unstack`).
    
  

In [53]:
df_us

,RACE_NUMBER,FIRST_NAME,SURNAME,GENDER,GENDER_POSITION,CATEGORY,CATEGORY_POSITION,CLUB,5K,10K,...,min_per_km_11_15,min_per_km_16_20,min_per_km_21_FINISH,5K_10K_SECTION_TIME,10K_15K_SECTION_TIME,15K_20K_SECTION_TIME,Month,0K_5K_SECTION_TIME,ANALYSIS_CATEGORY,Label
4813,2338,John,Steedman,Male,2646,M45+,361,NaN,1900-01-01 00:24:30,1900-01-01 00:51:55,...,5.650000,6.160000,5.861807,27.416667,28.250000,30.800000,Apr,24.500000,Us,John
7601,13475,Ellena,Greenow,Female,3884,F40+,636,NaN,1900-01-01 00:36:25,1900-01-01 01:14:22,...,7.986667,8.220000,7.152620,37.950000,39.933333,41.100000,Apr,36.416667,Us,Ellena
2761,3371,John,Steedman,Male,1887,M50+,151,NaN,1900-01-01 00:23:49,1900-01-01 00:49:07,...,5.133333,5.716667,5.193622,25.300000,25.666667,28.583333,Oct,23.816667,Us,John
6001,15487,Ellena,Greenow,Female,2711,F40+,423,NaN,1900-01-01 00:29:50,1900-01-01 01:01:44,...,6.576667,6.800000,5.725133,31.900000,32.883333,34.000000,Oct,29.833333,Us,Ellena


In [58]:

lstSections = ['0K_5K_SECTION_TIME', '5K_10K_SECTION_TIME', '10K_15K_SECTION_TIME', '15K_20K_SECTION_TIME'  ]

df_us = df_us [ ['Label' ]   + lstSections  ] #, 'CHIP_TIME_MINS'

pd.melt(df_us, id_vars='Label', value_vars= lstSections)


,Label,variable,value
0,John,0K_5K_SECTION_TIME,24.500000
1,Ellena,0K_5K_SECTION_TIME,36.416667
2,John,0K_5K_SECTION_TIME,23.816667
3,Ellena,0K_5K_SECTION_TIME,29.833333
4,John,5K_10K_SECTION_TIME,27.416667
5,Ellena,5K_10K_SECTION_TIME,37.950000
6,John,5K_10K_SECTION_TIME,25.300000
7,Ellena,5K_10K_SECTION_TIME,31.900000
8,John,10K_15K_SECTION_TIME,28.250000
9,Ellena,10K_15K_SECTION_TIME,39.933333


In [28]:
# Get the required columns
df_us.loc [ ]

,RACE_NUMBER,FIRST_NAME,SURNAME,GENDER,GENDER_POSITION,CATEGORY,CATEGORY_POSITION,CLUB,5K,10K,...,min_per_km_11_15,min_per_km_16_20,min_per_km_21_FINISH,5K_10K_SECTION_TIME,10K_15K_SECTION_TIME,15K_20K_SECTION_TIME,Month,0K_5K_SECTION_TIME,ANALYSIS_CATEGORY,Label
4813,2338,John,Steedman,Male,2646,M45+,361,NaN,1900-01-01 00:24:30,1900-01-01 00:51:55,...,5.650000,6.160000,5.861807,27.416667,28.250000,30.800000,Apr,24.500000,Us,John
7601,13475,Ellena,Greenow,Female,3884,F40+,636,NaN,1900-01-01 00:36:25,1900-01-01 01:14:22,...,7.986667,8.220000,7.152620,37.950000,39.933333,41.100000,Apr,36.416667,Us,Ellena
2761,3371,John,Steedman,Male,1887,M50+,151,NaN,1900-01-01 00:23:49,1900-01-01 00:49:07,...,5.133333,5.716667,5.193622,25.300000,25.666667,28.583333,Oct,23.816667,Us,John
6001,15487,Ellena,Greenow,Female,2711,F40+,423,NaN,1900-01-01 00:29:50,1900-01-01 01:01:44,...,6.576667,6.800000,5.725133,31.900000,32.883333,34.000000,Oct,29.833333,Us,Ellena


In [20]:
# Now Find the means for each section and Category
df_agg1 =   df.groupby (['ANALYSIS_CATEGORY' , 'GENDER', 'Month' ] ).agg (
     num_runners=('RACE_NUMBER', 'count' )
    ,time_mins = ('CHIP_TIME_MINS', 'mean' )
).reset_index(drop=False)
df_agg1.sort_values('ANALYSIS_CATEGORY')

,ANALYSIS_CATEGORY,GENDER,Month,num_runners,time_mins
0,Elite,Female,Apr,146,92.481164
1,Elite,Female,Oct,204,90.855801
2,Elite,Male,Apr,488,82.655977
3,Elite,Male,Oct,627,81.603402
4,Normal,Female,Apr,5030,143.687247
5,Normal,Female,Oct,5516,144.383883
6,Normal,Male,Apr,5906,123.961147
7,Normal,Male,Oct,6809,125.796245
8,Us,Female,Apr,1,163.250000
9,Us,Female,Oct,1,134.900000


#### Pivot

In [84]:
df_piv = df_agg1.reset_index().pivot( index=['ANALYSIS_CATEGORY', 'GENDER'],
                                      columns=['Month'], 
                                      values=['time_mins'] ).fillna(0).reset_index (drop=False)
df_piv

ANALYSIS_CATEGORY  GENDER   time_mins            
Month                                   Apr         Oct
0                 Elite  Female   92.481164   90.855801
1                 Elite    Male   82.655977   81.603402
2                Normal  Female  143.687247  144.383883
3                Normal    Male  123.961147  125.796245
4                    Us  Female  163.250000  134.900000
5                    Us    Male  117.400000  109.066667

In [17]:
# Useful function 
# https://stackoverflow.com/questions/14507794/how-to-flatten-a-hierarchical-index-in-columns

def flattenHierarchicalCol(col,sep = '_'):
    if not type(col) is tuple:
        return col
    else:
        new_col = ''
        for leveli,level in enumerate(col):
            if not level == '':
                if not leveli == 0:
                    new_col += sep
                new_col += level
        return new_col
    
df_piv.columns = df_piv.columns.map (flattenHierarchicalCol )

df_piv

NameError: name 'df_piv' is not defined

#### Improve the labelling

In [88]:
df_piv['Label'] = df_piv.ANALYSIS_CATEGORY
df_piv.loc[(df_piv.Label == 'Us') & (df_piv.GENDER == 'Male') , 'Label'] = 'John'
df_piv.loc[(df_piv.Label == 'Us') & (df_piv.GENDER == 'Female') , 'Label'] = 'Ellena'
df_piv

,ANALYSIS_CATEGORY,GENDER,time_mins_Apr,time_mins_Oct,Label
0,Elite,Female,92.481164,90.855801,Elite
1,Elite,Male,82.655977,81.603402,Elite
2,Normal,Female,143.687247,144.383883,Normal
3,Normal,Male,123.961147,125.796245,Normal
4,Us,Female,163.250000,134.900000,Ellena
5,Us,Male,117.400000,109.066667,John


In [89]:
df_piv.to_csv ( '..\Data\Staging\chart1_data.csv', index=False )